### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [84]:
import pandas as pd
import urllib.request
import json 
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from urllib import request, parse

def get_prediction(field_array=None, myurl=None):
    """
    Input parameters:
        field_array: python list - contains raw fields needed to make a prediction
        myurl: REST API url
    Returns:
        result: float - model prediction
    """
    result = 0

    body = {'msg': field_array.to_json()} 

    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

API_URL = "http://localhost:8180/predict"

X_test = pd.read_csv("X_test_spam.csv")
y_test = pd.read_csv("y_test_spam.csv")

Посмотрим что за данные у нас есть

In [56]:
X_test[['msg']].tail(3)

,msg
13641,не очень понимаю двух вещей.\n1. Масок в прода...
13642,На тропинке в лесу какой-то небезразличный муж...
13643,А какой телефон участкового?


### Getting predictions from REST API

In [85]:
%%time
predictions = X_test[['msg']].iloc[:500].apply(lambda x: get_prediction(x, API_URL), 1)

Wall time: 2.74 s


check performance (roc auc)

In [86]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

1.0